In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

In [2]:
det_cats = {
    'pz14' : pd.read_csv('/home/rt2122/Data/scans_old/scans_extended/connected/ep14_thr0.1.csv'),
    'pz20' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz20_thr0.1_step8.csv'),
    'pz25' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz25_thr0.1_step8.csv'),
    'pz40' : pd.read_csv('/home/rt2122/Data/scans_old/scans_extended/connected/ep40_thr0.1.csv'),
    'pz_act10' : pd.read_csv(
    '/home/rt2122/Data/scans_old/scans_extended/connected/full_act_cut_ep10_thr0.1_step8.csv'),
    'pz_act14' : pd.read_csv(
    '/home/rt2122/Data/scans_old/scans_extended/connected/full_act_cut_ep14_thr0.1_step8.csv'),
    'pz_act20' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz_act20_thr0.1_step8.csv'),
    'pz_act25' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz_act25_thr0.1_step8.csv')
}

In [3]:
psz2 = None
with fits.open('/home/rt2122/Data/original_catalogs/psz2.fits') as hdul:
    data = hdul[1].data
    psz2 = Table(data).to_pandas()
mcxc = None
with fits.open('/home/rt2122/Data/original_catalogs/mcxc.fits') as hdul:
    data = hdul[1].data
    mcxc = Table(data).to_pandas()
rm = None
with fits.open('/home/rt2122/Data/original_catalogs/redmapper.fits.gz') as hdul:
    data = Table(hdul[1].data)
    names = [name for name in data.colnames if len(data[name].shape) <= 1]
    rm = data[names].to_pandas()
act = None
with fits.open('/home/rt2122/Data/original_catalogs/act.fits') as hdul:
    data = hdul[1].data
    act = Table(data).to_pandas()

In [4]:
true_cats = {'psz2' : psz2, 'mcxc' : mcxc, 'rm' : rm, 'act' : act}

In [5]:
true_cats['mcxc'].rename({'RAdeg' : 'RA', 'DEdeg' : 'DEC'}, axis='columns', inplace=True)
true_cats['act'].rename({'RADeg' : 'RA', 'decDeg' : 'DEC'}, axis='columns', inplace=True)

In [6]:
match_dist = 5 / 60

In [9]:
comp_df = []
recall_df = []
for det_name in det_cats:
    line = {}
    line_r = {}
    
    sc = SkyCoord(ra=np.array(det_cats[det_name]['RA'])*u.degree, 
                  dec=np.array(det_cats[det_name]['DEC'])*u.degree, frame='icrs')
    
    for tr_name in true_cats: 
        tr_sc = SkyCoord(ra=np.array(true_cats[tr_name]['RA'])*u.degree, 
                      dec=np.array(true_cats[tr_name]['DEC'])*u.degree, frame='icrs')
        idx, d2d, _ = sc.match_to_catalog_sky(tr_sc)
        matched = d2d.degree <= match_dist
        line[tr_name] = np.count_nonzero(det_cats[det_name].iloc[matched]['status'] != 'fn')
        line[tr_name+'_err'] = calc_error(det_cats[det_name], true_cats[tr_name])
        line[tr_name+'_clear'] = line[tr_name] - line[tr_name+'_err']
        line_r[tr_name] = line[tr_name] / len(true_cats[tr_name])
    line['fp'] = np.count_nonzero(det_cats[det_name]['status'] == 'fp')
    line_r['fp'] = line['fp']
    comp_df.append(pd.DataFrame(line, index=[det_name]))
    recall_df.append(pd.DataFrame(line_r, index=[det_name]))

line = {}
for tr_name in true_cats:
    line[tr_name] = len(true_cats[tr_name])
    line[tr_name+'_err'] = 0
line['fp'] = 0
comp_df.append(pd.DataFrame(line, index=['all']))
comp_df = pd.concat(comp_df)
recall_df = pd.concat(recall_df)

In [10]:
comp_df

,psz2,psz2_err,psz2_clear,mcxc,mcxc_err,mcxc_clear,rm,rm_err,rm_clear,act,act_err,act_clear,fp
pz14,1491,22.4,1468.6,725,23.1,701.9,1242,399.4,842.6,849,70.5,778.5,15828
pz20,1528,23.6,1504.4,740,25.5,714.5,1306,406.6,899.4,875,72.2,802.8,23104
pz25,1525,25.5,1499.5,747,27.3,719.7,1374,448.2,925.8,888,75.2,812.8,20611
pz40,1506,22.5,1483.5,739,23.5,715.5,1279,427.8,851.2,871,72.4,798.6,17306
pz_act10,1361,17.3,1343.7,659,18.8,640.2,1029,283.9,745.1,926,46.6,879.4,16316
pz_act14,1363,18.0,1345.0,670,19.0,651.0,1211,334.2,876.8,1282,57.2,1224.8,16484
pz_act20,1217,15.4,1201.6,599,15.9,583.1,1182,261.2,920.8,1866,47.8,1818.2,9398
pz_act25,1260,19.6,1240.4,610,17.4,592.6,1271,287.8,983.2,2152,58.0,2094.0,15275
all,1653,0.0,NaN,1743,0.0,NaN,26111,0.0,NaN,4195,0.0,NaN,0


In [11]:
recall_df

,psz2,mcxc,rm,act,fp
pz14,0.901996,0.415950,0.047566,0.202384,15828
pz20,0.924380,0.424555,0.050017,0.208582,23104
pz25,0.922565,0.428571,0.052622,0.211681,20611
pz40,0.911071,0.423982,0.048983,0.207628,17306
pz_act10,0.823351,0.378084,0.039409,0.220739,16316
pz_act14,0.824561,0.384395,0.046379,0.305602,16484
pz_act20,0.736237,0.343660,0.045268,0.444815,9398
pz_act25,0.762250,0.349971,0.048677,0.512992,15275


In [8]:
def calc_error(det_cat, true_cat, shift=15/60, match_dist=5/60, n_try=10, seed=0):
    import numpy as np
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    
    error = []
    np.random.seed(seed)
    for i in range(n_try):
        det_sc = SkyCoord(ra=np.array(det_cat['RA']) * u.degree, 
                          dec=np.array(det_cat['DEC']) * u.degree, frame='icrs')
        angles = np.random.randint(0, 360, len(det_cat))
        det_sc = det_sc.directional_offset_by(angles*u.degree, shift)

        true_sc = SkyCoord(ra=np.array(true_cat['RA']) * u.degree, 
                           dec=np.array(true_cat['DEC']) * u.degree, frame='icrs')
        _, d2d, _ = det_sc.match_to_catalog_sky(true_sc)
        c_error = np.count_nonzero(d2d.degree < match_dist)
        error.append(c_error)
    return np.array(error).mean()